# CNS Conservatory Tree generation

## STEP 1: Generate separate files for every unique ortholog in list

In [1]:
import pandas as pd
import glob
import os
# set family name
family = "Malvaceae"
os.chdir(family) #this navigates the directory to the family folder
!ls # list folder contents

Family_gene_table.csv		Malvaceae_PeptideLibraries
FinalConcatenated		Malvaceae_species.csv
LociForAlignment		PhylipAlignments_Attempt1
LociForAlignment_Attempt1	Test
LocusLists			Test_Attempt1
LocusLists_Attempt1		uniqueSpeciesNicknames.csv
Malvaceae_CRE_inall.csv		uniqueSpeciesNicknames_Attempt1
Malvaceae_CRE_inall_unique.csv


In [2]:
df = pd.read_csv(family + '_CRE_inall_unique.csv') # import csv with pandas and display
df

,RefGeneID,Species,SpeciesGeneId,UpstreamScore,DownstreamScore,ProteinScore,uniqueSpecies
0,Graim_XP_012434182,Ccapsularis,OMO98109,23179,0,368,Ccapsularis
1,Graim_XP_012434182,Colitorius.fixed,Corc_OMP03267,22797,0,366,Colitorius.fixed
2,Graim_XP_012434182,Dzibethinus.fixed,Dzib_XP_022746767,15204,0,368,Dzibethinus.fixed
3,Graim_XP_012434182,Garboreum.fixed,Ga11G0851,785788,0,390,Garboreum.fixed
4,Graim_XP_012434182,Garidum,MBA0687616,4474,0,303,Garidum
...,...,...,...,...,...,...,...
35335,Graim_XP_012493033,Gtrilobum.fixed,Gtri_MBA0758428,100017,46611,800,Gtrilobum.fixed
35336,Graim_XP_012493033,Hcannabinus,GWHPACDB033337,6420,3257,800,Hcannabinus
35337,Graim_XP_012493033,Hsyriacus,KAE8699551,10093,0,800,Hsyriacus
35338,Graim_XP_012493033,Humbratica.fixed,Hum_XP_021275212,19012,9094,800,Humbratica.fixed


In [3]:
df = df[["RefGeneID", "SpeciesGeneId", "uniqueSpecies"]] # just keep relevant columns
df

,RefGeneID,SpeciesGeneId,uniqueSpecies
0,Graim_XP_012434182,OMO98109,Ccapsularis
1,Graim_XP_012434182,Corc_OMP03267,Colitorius.fixed
2,Graim_XP_012434182,Dzib_XP_022746767,Dzibethinus.fixed
3,Graim_XP_012434182,Ga11G0851,Garboreum.fixed
4,Graim_XP_012434182,MBA0687616,Garidum
...,...,...,...
35335,Graim_XP_012493033,Gtri_MBA0758428,Gtrilobum.fixed
35336,Graim_XP_012493033,GWHPACDB033337,Hcannabinus
35337,Graim_XP_012493033,KAE8699551,Hsyriacus
35338,Graim_XP_012493033,Hum_XP_021275212,Humbratica.fixed


In [4]:
df = df.pivot(index='RefGeneID', columns='uniqueSpecies', values='SpeciesGeneId')
df

uniqueSpecies,Ccapsularis,Colitorius.fixed,Dzibethinus.fixed,Garboreum.fixed,Garidum,Garmourianum.fixed,Gbarbadense,Gdarwinii.fixed,Ghirsutum.fixed,Glaxum.fixed,Globatum.fixed,Gmustelinum.fixed,Gschwendimanii.fixed,Gtomentosum.fixed,Gtrilobum.fixed,Hcannabinus,Hsyriacus,Humbratica.fixed,Tcacao.fixed
RefGeneID,,,,,,,,,,,,,,,,,,,
Graim_XP_012434182,OMO98109,Corc_OMP03267,Dzib_XP_022746767,Ga11G0851,MBA0687616,Garm_MBA0844122,KAB2059224,Gdar_TYG47239,Ghir_XP_016713807,Glax_MBA0716448,Glob_MBA0561520,Gmus_TYI57860,Gsch_MBA0877401,Gtom_TYI03050,Gtri_MBA0771200,GWHPACDB052070,KAE8684095,Hum_XP_021283339,Tca_XP_007031019
Graim_XP_012434237,OMO96706,Corc_OMP10962,Dzib_XP_022742257,Ga11G0820,MBA0687425,Garm_MBA0833113,KAB2059266,Gdar_TYG96056,Ghir_XP_016666795,Glax_MBA0716467,Glob_MBA0577630,Gmus_TYI57903,Gsch_MBA0859677,Gtom_TYH46323,Gtri_MBA0770968,GWHPACDB012579,KAE8725519,Hum_XP_021283105,Tca_XP_007031089
Graim_XP_012434271,OMO86269,Corc_OMP10937,Dzib_XP_022720140,Ga11G0787,MBA0687445,Garm_MBA0827142,KAB2005993,Gdar_TYG47309,Ghir_XP_016713750,Glax_MBA0716486,Glob_MBA0561556,Gmus_TYI57936,Gsch_MBA0875559,Gtom_TYH46363,Gtri_MBA0785407,GWHPACDB006865,KAE8660192,Hum_XP_021283453,Tca_XP_017976052
Graim_XP_012434276,OMO86276,Corc_OMP09380,Dzib_XP_022745699,Ga11G0783,MBA0687447,Garm_MBA0827140,KAB2005997,Gdar_TYG47313,Ghir_XP_016713685,Glax_MBA0710010,Glob_MBA0554861,Gmus_TYI57941,Gsch_MBA0853958,Gtom_TYH46367,Gtri_MBA0770985,GWHPACDB009641,KAE8685020,Hum_XP_021281551,Tca_XP_007031154
Graim_XP_012434347,OMO65826,Corc_OMO98911,Dzib_XP_022735022,Ga11G0736,MBA0699451,Garm_MBA0843808,KAB2006049,Gdar_TYG47362,Ghir_XP_016713768,Glax_MBA0728125,Glob_MBA0553688,Gmus_TYI57996,Gsch_MBA0874934,Gtom_TYH46432,Gtri_MBA0783521,GWHPACDB019067,KAE8698369,Hum_XP_021287486,Tca_XP_017980671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Graim_XP_012492939,OMP05406,Corc_OMO98963,Dzib_XP_022719259,Ga11G0970,MBA0687323,Garm_MBA0833010,KAB2005752,Gdar_TYG47072,Ghir_XP_016721649,Glax_MBA0716364,Glob_MBA0561427,Gmus_TYI57695,Gsch_MBA0859936,Gtom_TYH46088,Gtri_MBA0770821,GWHPACDB036530,KAE8734694,Hum_XP_021282552,Tca_XP_007030700
Graim_XP_012492941,OMO62332,Corc_OMO98961,Dzib_XP_022719262,Ga11G0968,MBA0687325,Garm_MBA0833012,KAB2005755,Gdar_TYG47074,Ghir_XP_016668160,Glax_MBA0716366,Glob_MBA0561428,Gmus_TYI57698,Gsch_MBA0860580,Gtom_TYH46092,Gtri_MBA0770823,GWHPACDB051674,KAE8729096,Hum_XP_021283107,Tca_XP_007030702
Graim_XP_012492958,OMO62323,Corc_OMO98948,Dzib_XP_022719273,Ga11G0958,MBA0699223,Garm_MBA0843596,KAB2005767,Gdar_TYG47144,Ghir_XP_016667992,Glax_MBA0716373,Glob_MBA0561437,Gmus_TYI57711,Gsch_MBA0873469,Gtom_TYH46104,Gtri_MBA0770828,GWHPACDB051680,KAE8695444,Hum_XP_021283093,Tca_XP_007030715


In [14]:
df.to_csv('Family_gene_table.csv') #write new table to csv
!ls # list dir contents

Family_gene_table.csv  locus_1416.txt  locus_1836.txt  locus_582.txt
locus_0.txt	       locus_1417.txt  locus_1837.txt  locus_583.txt
locus_1.txt	       locus_1418.txt  locus_1838.txt  locus_584.txt
locus_10.txt	       locus_1419.txt  locus_1839.txt  locus_585.txt
locus_100.txt	       locus_142.txt   locus_184.txt   locus_586.txt
locus_1000.txt	       locus_1420.txt  locus_1840.txt  locus_587.txt
locus_1001.txt	       locus_1421.txt  locus_1841.txt  locus_588.txt
locus_1002.txt	       locus_1422.txt  locus_1842.txt  locus_589.txt
locus_1003.txt	       locus_1423.txt  locus_1843.txt  locus_59.txt
locus_1004.txt	       locus_1424.txt  locus_1844.txt  locus_590.txt
locus_1005.txt	       locus_1425.txt  locus_1845.txt  locus_591.txt
locus_1006.txt	       locus_1426.txt  locus_1846.txt  locus_592.txt
locus_1007.txt	       locus_1427.txt  locus_1847.txt  locus_593.txt
locus_1008.txt	       locus_1428.txt  locus_1848.txt  locus_594.txt
locus_1009.txt	       locus_1429.txt  locus_1849.txt  loc

In [6]:
# split Family_gene_table.csv into separate files - one file for each gene in the table.
!split -l 1 -a 3 Family_gene_table.csv locus_ 

In [7]:
!rm locus_aaa #remove the txt file created from the header

In [8]:
# check that the number of files matches the number of genes. Pandas does not count the header as a row in the dataframe.
if (len(df) != len(glob.glob("locus_*"))):
    print("Problem: csv rownumber and generated file number do not match. ", len(df), ", ", len(glob.glob("locus_*")))
else:
    print("Same number: ", len(df), ", ", len(glob.glob("locus_*")))

Same number:  1860 ,  1860


In [9]:
# move all files to a new directory - 'LocusLists' within family folder
!mkdir LocusLists
!mv locus_* LocusLists

In [10]:
#change directories to LocusLists and check
os.chdir('LocusLists')
!ls

locus_aab  locus_akh  locus_aun  locus_bet  locus_boz  locus_bzf  locus_cjl
locus_aac  locus_aki  locus_auo  locus_beu  locus_bpa  locus_bzg  locus_cjm
locus_aad  locus_akj  locus_aup  locus_bev  locus_bpb  locus_bzh  locus_cjn
locus_aae  locus_akk  locus_auq  locus_bew  locus_bpc  locus_bzi  locus_cjo
locus_aaf  locus_akl  locus_aur  locus_bex  locus_bpd  locus_bzj  locus_cjp
locus_aag  locus_akm  locus_aus  locus_bey  locus_bpe  locus_bzk  locus_cjq
locus_aah  locus_akn  locus_aut  locus_bez  locus_bpf  locus_bzl  locus_cjr
locus_aai  locus_ako  locus_auu  locus_bfa  locus_bpg  locus_bzm  locus_cjs
locus_aaj  locus_akp  locus_auv  locus_bfb  locus_bph  locus_bzn  locus_cjt
locus_aak  locus_akq  locus_auw  locus_bfc  locus_bpi  locus_bzo  locus_cju
locus_aal  locus_akr  locus_aux  locus_bfd  locus_bpj  locus_bzp  locus_cjv
locus_aam  locus_aks  locus_auy  locus_bfe  locus_bpk  locus_bzq  locus_cjw
locus_aan  locus_akt  locus_auz  locus_bff  locus_bpl  locus_bzr  locus_cjx
locus_aao  l

In [11]:
# Rename all the resulting gene files with a number and a .txt extension
!n=1;
!for f in *; do mv "$f" "locus_$((n++)).txt"; done
!ls

locus_0.txt	locus_1332.txt	locus_1668.txt	locus_329.txt  locus_664.txt
locus_1.txt	locus_1333.txt	locus_1669.txt	locus_33.txt   locus_665.txt
locus_10.txt	locus_1334.txt	locus_167.txt	locus_330.txt  locus_666.txt
locus_100.txt	locus_1335.txt	locus_1670.txt	locus_331.txt  locus_667.txt
locus_1000.txt	locus_1336.txt	locus_1671.txt	locus_332.txt  locus_668.txt
locus_1001.txt	locus_1337.txt	locus_1672.txt	locus_333.txt  locus_669.txt
locus_1002.txt	locus_1338.txt	locus_1673.txt	locus_334.txt  locus_67.txt
locus_1003.txt	locus_1339.txt	locus_1674.txt	locus_335.txt  locus_670.txt
locus_1004.txt	locus_134.txt	locus_1675.txt	locus_336.txt  locus_671.txt
locus_1005.txt	locus_1340.txt	locus_1676.txt	locus_337.txt  locus_672.txt
locus_1006.txt	locus_1341.txt	locus_1677.txt	locus_338.txt  locus_673.txt
locus_1007.txt	locus_1342.txt	locus_1678.txt	locus_339.txt  locus_674.txt
locus_1008.txt	locus_1343.txt	locus_1679.txt	locus_34.txt   locus_675.txt
locus_1009.txt	locus_1344.txt	locus_168.txt	locus_

In [12]:
# each file needs commas replaced with /n, " characters removed, and > added to the beginning of every line
!for file in *.txt; do sed 's/,/\n/g' "$file" > "$file.1"; sed 's/^/\>/' "$file.1" > "$file.2"; sed 's/^M//g' "$file.2" > "$file.3" ; done

# get rid of backup files
!rm *.txt
!rm *.txt.1
!rm *.txt.2

# remove .3 extension on final file
!for file in *.txt.3; do mv -- "$file" "${file%.txt.3}.txt"; done

# check that every file has the right number of taxa (refer to species list)
!grep -c -F ">" *.txt > taxon_counts.txt

In [ ]:
# optional: sometimes adding the ">" at the beginning of each txt file causes no match to protein library. Removed for Fabaceae.
!for file in *.txt; do sed -i 's/[>]//g' $file; done

In [13]:
# check that everything looks good by looking at some random files and checking counts
!cat locus_1.txt
!cat locus_888.txt

!cat taxon_counts.txt

>Graim_XP_012434237
>OMO96706
>Corc_OMP10962
>Dzib_XP_022742257
>Ga11G0820
>MBA0687425
>Garm_MBA0833113
>KAB2059266
>Gdar_TYG96056
>Ghir_XP_016666795
>Glax_MBA0716467
>Glob_MBA0577630
>Gmus_TYI57903
>Gsch_MBA0859677
>Gtom_TYH46323
>Gtri_MBA0770968
>GWHPACDB012579
>KAE8725519
>Hum_XP_021283105
>Tca_XP_007031089
>Graim_XP_012463532
>OMO78385
>Corc_OMP06189
>Dzib_XP_022758352
>Ga07G2437
>MBA0699085
>Garm_MBA0843443
>KAB2022828
>Gdar_TYG62748
>Ghir_XP_016724988
>Glax_MBA0727740
>Glob_MBA0573475
>Gmus_TYI75014
>Gsch_MBA0874796
>Gtom_TYH64264
>Gtri_MBA0783100
>GWHPACDB044783
>KAE8686170
>Hum_XP_021279662
>Tca_XP_007050397
locus_0.txt:20
locus_1.txt:20
locus_10.txt:20
locus_100.txt:20
locus_1000.txt:20
locus_1001.txt:20
locus_1002.txt:20
locus_1003.txt:20
locus_1004.txt:20
locus_1005.txt:20
locus_1006.txt:20
locus_1007.txt:20
locus_1008.txt:20
locus_1009.txt:20
locus_101.txt:20
locus_1010.txt:20
locus_1011.txt:20
locus_1012.txt:20
locus_1013.txt:20
locus_1014.txt:20
locus_1015.txt:20
locus_10

In [15]:
#move files to peptide library folder
#!mkdir ../Rosaceae_PeptideLibraries
!cp locus_* ../Malvaceae_PeptideLibraries

## STEP 2: Search for all loci in peptide fasta files

### Transferring files directly from the Marlin server to the UMass cluster  

* Step 1: On the UMass cluster terminal, connect to Marlin using sftp and navigate to peptide folder

`sftp username@marlin.bio.umass.edu`  
`cd ../../../data/bartlett/Conservatory/peptides`  
`ls` to list all family folders  
`cd Fabaceae` #example folder with peptide files on Marlin  

* Step 2: Set the local directory where you want to download the files using the lcd command  

note: lcd works exactly like cd. If you are unsure where you are in your local directory, you can us lls, which works the same as ls.  
`lcd CNS/Fabaceae/Fabaceae_PeptideLibraries` #example local destination folder on UMass cluster

* Step 3: Copy everything over using mget

`mget *` #transfers over each file in the directory


In [18]:
# change directory to peptide library
os.chdir('../Malvaceae_PeptideLibraries')
!ls
#os.chdir(family + "_PeptideLibraries")

Attempt1			     locus_1543.txt  locus_435.txt
Bceiba.fixed.proteins.fasta	     locus_1544.txt  locus_436.txt
Ccapsularis.proteins.fasta	     locus_1545.txt  locus_437.txt
Colitorius.fixed.proteins.fasta      locus_1546.txt  locus_438.txt
Dzibethinus.fixed.proteins.fasta     locus_1547.txt  locus_439.txt
Garboreum.fixed.proteins.fasta	     locus_1548.txt  locus_44.txt
Garidum.proteins.fasta		     locus_1549.txt  locus_440.txt
Garmourianum.fixed.proteins.fasta    locus_155.txt   locus_441.txt
Gbarbadense.proteins.fasta	     locus_1550.txt  locus_442.txt
Gdarwinii.fixed.proteins.fasta	     locus_1551.txt  locus_443.txt
Ghirsutum.fixed.proteins.fasta	     locus_1552.txt  locus_444.txt
Glaxum.fixed.proteins.fasta	     locus_1553.txt  locus_445.txt
Globatum.fixed.proteins.fasta	     locus_1554.txt  locus_446.txt
Gmustelinum.fixed.proteins.fasta     locus_1555.txt  locus_447.txt
Graimondii.fixed.proteins.fasta      locus_1556.txt  locus_448.txt
Gschwendimanii.fixed.proteins.fasta  locus_1557.

In [19]:
# convert fasta files to have all amino acids on 1 line
!ls
!for f in *.fasta; do dos2unix $f; done
!for f in *.fasta; do awk '/^>/ {printf("\n%s\n",$0);next; } { printf("%s",$0);} END {printf("\n");}' $f > $f.fullfile.1line.fa; done


Attempt1			     locus_1543.txt  locus_435.txt
Bceiba.fixed.proteins.fasta	     locus_1544.txt  locus_436.txt
Ccapsularis.proteins.fasta	     locus_1545.txt  locus_437.txt
Colitorius.fixed.proteins.fasta      locus_1546.txt  locus_438.txt
Dzibethinus.fixed.proteins.fasta     locus_1547.txt  locus_439.txt
Garboreum.fixed.proteins.fasta	     locus_1548.txt  locus_44.txt
Garidum.proteins.fasta		     locus_1549.txt  locus_440.txt
Garmourianum.fixed.proteins.fasta    locus_155.txt   locus_441.txt
Gbarbadense.proteins.fasta	     locus_1550.txt  locus_442.txt
Gdarwinii.fixed.proteins.fasta	     locus_1551.txt  locus_443.txt
Ghirsutum.fixed.proteins.fasta	     locus_1552.txt  locus_444.txt
Glaxum.fixed.proteins.fasta	     locus_1553.txt  locus_445.txt
Globatum.fixed.proteins.fasta	     locus_1554.txt  locus_446.txt
Gmustelinum.fixed.proteins.fasta     locus_1555.txt  locus_447.txt
Graimondii.fixed.proteins.fasta      locus_1556.txt  locus_448.txt
Gschwendimanii.fixed.proteins.fasta  locus_1557.

/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found
/bin/bash: dos2unix: command not found


In [21]:
# check that it worked
!ls
!head Bceiba.fixed.proteins.fasta
!head Bceiba.fixed.proteins.fasta.fullfile.1line.fa

Attempt1					       locus_1816.txt
Bceiba.fixed.proteins.fasta			       locus_1817.txt
Bceiba.fixed.proteins.fasta.fullfile.1line.fa	       locus_1818.txt
Ccapsularis.proteins.fasta			       locus_1819.txt
Ccapsularis.proteins.fasta.fullfile.1line.fa	       locus_182.txt
Colitorius.fixed.proteins.fasta			       locus_1820.txt
Colitorius.fixed.proteins.fasta.fullfile.1line.fa      locus_1821.txt
Dzibethinus.fixed.proteins.fasta		       locus_1822.txt
Dzibethinus.fixed.proteins.fasta.fullfile.1line.fa     locus_1823.txt
Garboreum.fixed.proteins.fasta			       locus_1824.txt
Garboreum.fixed.proteins.fasta.fullfile.1line.fa       locus_1825.txt
Garidum.proteins.fasta				       locus_1826.txt
Garidum.proteins.fasta.fullfile.1line.fa	       locus_1827.txt
Garmourianum.fixed.proteins.fasta		       locus_1828.txt
Garmourianum.fixed.proteins.fasta.fullfile.1line.fa    locus_1829.txt
Gbarbadense.proteins.fasta			       locus_183.txt
Gbarbadense.proteins.fasta.fullfile.1line.fa	       locus_1830.

>cei.Scaffold1.1_CDS
MASRSFVSVEASQRMAIGSLAVPFGSSYGSRLRVLALVIYPLRGSKLFQSGMIAKEETAM
RWVASVLQMYEFQTALDRLREMVAHQTACIQSLEGKLNKVKREMSKKVEDADTAEVETTA
TEEVGQGSPSQSGQGKRKEQASHEQVGRVEQGTPEEAFE*
>cei.Scaffold1.2_CDS
MERNDKKKVMLTWIMLMMVVVAQGQATAKSSDFQPDINIKDCEKMCYDSCGHSIFHKDFV
VCFGLCMRGCKNQPGQVVYNCTSVCVKSAIDSDPNAGI*
>cei.Scaffold1.3_CDS
MGSLMADRGLDFKVLPGSGRLNVLVPEDHVVRQQKGCQVVRARRDLTSELSAESSSPSRL
SSSSQNDDLSEGSKGMGKGVCERNGLFKGMDASSGGQAVNNVKGVLIFRRNKIYLAILVV

>cei.Scaffold1.1_CDS
MASRSFVSVEASQRMAIGSLAVPFGSSYGSRLRVLALVIYPLRGSKLFQSGMIAKEETAMRWVASVLQMYEFQTALDRLREMVAHQTACIQSLEGKLNKVKREMSKKVEDADTAEVETTATEEVGQGSPSQSGQGKRKEQASHEQVGRVEQGTPEEAFE*
>cei.Scaffold1.2_CDS
MERNDKKKVMLTWIMLMMVVVAQGQATAKSSDFQPDINIKDCEKMCYDSCGHSIFHKDFVVCFGLCMRGCKNQPGQVVYNCTSVCVKSAIDSDPNAGI*
>cei.Scaffold1.3_CDS
MGSLMADRGLDFKVLPGSGRLNVLVPEDHVVRQQKGCQVVRARRDLTSELSAESSSPSRLSSSSQNDDLSEGSKGMGKGVCERNGLFKGMDASSGGQAVNNVKGVLIFRRNKIYLAILVVHQHPRGCQLGNQLWRSLW*
>cei.Scaffold1.4_CDS
MEHPDLTDVTEEEVETTATEGVGRGSPNQSGQGKKNEQASHEQVGRVKQGTPEEASE*
>cei.S

In [22]:
#removed -w for whole match
#for each locus text file, match the gene name to the protein sequence in the fasta file. Copy the ammino acids over.
!for f in *.txt; do grep -A 1 -m 1 -f $f -h *.fa > $f.fasta; done

In [23]:
#Identify genes with correct number of matches and copy good loci to a new folder. Change "target" variable number.
os.mkdir("GoodLoci")

In [25]:
%%script bash
    target=20
    allfiles=$(ls *.txt.fasta | wc -l)
    wrong=0
    echo "Total files: $allfiles"
    for f in *.txt.fasta
    do
        count=$(grep -o '>' $f | wc -l)
        if [ $count != $target ]
        then
            wrong=$((wrong + 1))
        else
        cp $f GoodLoci
        fi
    done
echo "Wrong: $wrong"
echo "Good with $target matches: $((allfiles - wrong))"

Total files: 1860
Wrong: 0
Good with 20 matches: 1860


In [38]:
#os.chdir('Malvaceae')
!mkdir LociForAlignment
!ls

Family_gene_table.csv	   Malvaceae_CRE_inall_unique.csv
FinalConcatenated	   Malvaceae_PeptideLibraries
LociForAlignment	   Malvaceae_species.csv
LociForAlignment_Attempt1  PhylipAlignments_Attempt1
LocusLists		   Test_Attempt1
LocusLists_Attempt1	   uniqueSpeciesNicknames_Attempt1
Malvaceae_CRE_inall.csv


In [39]:
os.chdir("Malvaceae_PeptideLibraries/GoodLoci")
!mv *.txt.fasta ../../LociForAlignment/

In [40]:
os.chdir('../../LociForAlignment')
!ls

locus_0.txt.fasta     locus_1556.txt.fasta  locus_440.txt.fasta
locus_1.txt.fasta     locus_1557.txt.fasta  locus_441.txt.fasta
locus_10.txt.fasta    locus_1558.txt.fasta  locus_442.txt.fasta
locus_100.txt.fasta   locus_1559.txt.fasta  locus_443.txt.fasta
locus_1000.txt.fasta  locus_156.txt.fasta   locus_444.txt.fasta
locus_1001.txt.fasta  locus_1560.txt.fasta  locus_445.txt.fasta
locus_1002.txt.fasta  locus_1561.txt.fasta  locus_446.txt.fasta
locus_1003.txt.fasta  locus_1562.txt.fasta  locus_447.txt.fasta
locus_1004.txt.fasta  locus_1563.txt.fasta  locus_448.txt.fasta
locus_1005.txt.fasta  locus_1564.txt.fasta  locus_449.txt.fasta
locus_1006.txt.fasta  locus_1565.txt.fasta  locus_45.txt.fasta
locus_1007.txt.fasta  locus_1566.txt.fasta  locus_450.txt.fasta
locus_1008.txt.fasta  locus_1567.txt.fasta  locus_451.txt.fasta
locus_1009.txt.fasta  locus_1568.txt.fasta  locus_452.txt.fasta
locus_101.txt.fasta   locus_1569.txt.fasta  locus_453.txt.fasta
locus_1010.txt.fasta  locus_157.txt.fasta

In [41]:
#remove '--' marks from grep
!for file in *; do sed -i 's/--//g' $file; done

In [42]:
#check that it worked (no '--')
!cat locus_1.txt.fasta

>OMO96706.1 Drought-responsive family protein [Corchorus capsularis]
MDAESWSARLSSASKRYQSALQSRSDMFMGFEEIDGEDDIREEFPCPFCSEYFDIVGLCCHIDDEHPVEAKNGRKRKSRKGGSHSTLSLLRKELREGNLQSLFGGSSCIMSSSNSAPDPLLSSFILPMVDDFVSVQPHFSSETSTTKKSADVNKSERNVQASPLSVKDQEEKAKRCEFVQGLLLSTILDDSL

>Corc_OMP10962.1 Drought responsive protein [Corchorus olitorius]
MDAESWSARLSSASKRYQSALQSRSDMFMGFEEIDGEDDIREEFPCPFCSEYFDIVGLCCHIDDEHPVEAKNGVCPVCAMRVGVDMRKRKSRKGGSHSTLSLLRKELREGNLQSLFGGSSCIMSSSNSAPDPLLSSFILPMVDDFVSVQPHFSSETSTTKKSADVNKSERNVQASPPLSVKDQEEKAKRCEFVQGLLLSTILDDSL

>Dzib_XP_022742257.1 protein DEHYDRATION-INDUCED 19 homolog 3-like [Durio zibethinus]
MDADSWSARLSSSSQRYQSALQSRTDMFMAFEEIDGEDNIREEFPCPFCSEYFDIVGLCCHIDDEHPVEAKNGVCPVCSMRVGVDMVAHITLQHGNIFKMQRKRKSHKGASHSILSLLKKELREGNLQSLFGGSSYIVSSSNSAPDPLLSSFILPIVDDFVSVQPPFSSETGTTKRSSNVNKSERNVESSPLAVKDQEEKAKRCEFVQRLLLSTVLDDIL

>Ga11G0820_CDS
MDADPWTLCLSSTARRYQSALQSRSDMFMGFEEMDGEDDVREEFPCPFCSEYFDIVGLCCHIDDEHPVEANNGVCPVCAMRVGVDMVAHITQQHGHIFKRKRRSRRSGSHSTLSLLRKELRDGNLQSLFGGSSCIAPSSN

## STEP 3: align using MAFFT & replace gene names with species names

#### Step 1: Start a new interactive session from the terminal. Make sure you are in the LociForAlignment directory.

`bsub -Is -q interactive -W 8:00 -n 1 -R "span[hosts=1]" bash`

#### Step 2: Run mafft on all fasta files in the LociForAlignment directory

`module load MAFFT/7.313`

`for f in *.fasta; do mafft-linsi $f > $f.aln; done`

### While MAFFT is running, identify and prep gene ids for species name replacement

In [43]:
#go to species family directory
os.chdir("../")
!ls

Family_gene_table.csv	   Malvaceae_CRE_inall_unique.csv
FinalConcatenated	   Malvaceae_PeptideLibraries
LociForAlignment	   Malvaceae_species.csv
LociForAlignment_Attempt1  PhylipAlignments_Attempt1
LocusLists		   Test_Attempt1
LocusLists_Attempt1	   uniqueSpeciesNicknames_Attempt1
Malvaceae_CRE_inall.csv


In [2]:
df = pd.read_csv("Family_gene_table.csv")
df

,RefGeneID,Ccapsularis,Colitorius.fixed,Dzibethinus.fixed,Garboreum.fixed,Garidum,Garmourianum.fixed,Gbarbadense,Gdarwinii.fixed,Ghirsutum.fixed,Glaxum.fixed,Globatum.fixed,Gmustelinum.fixed,Gschwendimanii.fixed,Gtomentosum.fixed,Gtrilobum.fixed,Hcannabinus,Hsyriacus,Humbratica.fixed,Tcacao.fixed
0,Graim_XP_012434182,OMO98109,Corc_OMP03267,Dzib_XP_022746767,Ga11G0851,MBA0687616,Garm_MBA0844122,KAB2059224,Gdar_TYG47239,Ghir_XP_016713807,Glax_MBA0716448,Glob_MBA0561520,Gmus_TYI57860,Gsch_MBA0877401,Gtom_TYI03050,Gtri_MBA0771200,GWHPACDB052070,KAE8684095,Hum_XP_021283339,Tca_XP_007031019
1,Graim_XP_012434237,OMO96706,Corc_OMP10962,Dzib_XP_022742257,Ga11G0820,MBA0687425,Garm_MBA0833113,KAB2059266,Gdar_TYG96056,Ghir_XP_016666795,Glax_MBA0716467,Glob_MBA0577630,Gmus_TYI57903,Gsch_MBA0859677,Gtom_TYH46323,Gtri_MBA0770968,GWHPACDB012579,KAE8725519,Hum_XP_021283105,Tca_XP_007031089
2,Graim_XP_012434271,OMO86269,Corc_OMP10937,Dzib_XP_022720140,Ga11G0787,MBA0687445,Garm_MBA0827142,KAB2005993,Gdar_TYG47309,Ghir_XP_016713750,Glax_MBA0716486,Glob_MBA0561556,Gmus_TYI57936,Gsch_MBA0875559,Gtom_TYH46363,Gtri_MBA0785407,GWHPACDB006865,KAE8660192,Hum_XP_021283453,Tca_XP_017976052
3,Graim_XP_012434276,OMO86276,Corc_OMP09380,Dzib_XP_022745699,Ga11G0783,MBA0687447,Garm_MBA0827140,KAB2005997,Gdar_TYG47313,Ghir_XP_016713685,Glax_MBA0710010,Glob_MBA0554861,Gmus_TYI57941,Gsch_MBA0853958,Gtom_TYH46367,Gtri_MBA0770985,GWHPACDB009641,KAE8685020,Hum_XP_021281551,Tca_XP_007031154
4,Graim_XP_012434347,OMO65826,Corc_OMO98911,Dzib_XP_022735022,Ga11G0736,MBA0699451,Garm_MBA0843808,KAB2006049,Gdar_TYG47362,Ghir_XP_016713768,Glax_MBA0728125,Glob_MBA0553688,Gmus_TYI57996,Gsch_MBA0874934,Gtom_TYH46432,Gtri_MBA0783521,GWHPACDB019067,KAE8698369,Hum_XP_021287486,Tca_XP_017980671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,Graim_XP_012492939,OMP05406,Corc_OMO98963,Dzib_XP_022719259,Ga11G0970,MBA0687323,Garm_MBA0833010,KAB2005752,Gdar_TYG47072,Ghir_XP_016721649,Glax_MBA0716364,Glob_MBA0561427,Gmus_TYI57695,Gsch_MBA0859936,Gtom_TYH46088,Gtri_MBA0770821,GWHPACDB036530,KAE8734694,Hum_XP_021282552,Tca_XP_007030700
1856,Graim_XP_012492941,OMO62332,Corc_OMO98961,Dzib_XP_022719262,Ga11G0968,MBA0687325,Garm_MBA0833012,KAB2005755,Gdar_TYG47074,Ghir_XP_016668160,Glax_MBA0716366,Glob_MBA0561428,Gmus_TYI57698,Gsch_MBA0860580,Gtom_TYH46092,Gtri_MBA0770823,GWHPACDB051674,KAE8729096,Hum_XP_021283107,Tca_XP_007030702
1857,Graim_XP_012492958,OMO62323,Corc_OMO98948,Dzib_XP_022719273,Ga11G0958,MBA0699223,Garm_MBA0843596,KAB2005767,Gdar_TYG47144,Ghir_XP_016667992,Glax_MBA0716373,Glob_MBA0561437,Gmus_TYI57711,Gsch_MBA0873469,Gtom_TYH46104,Gtri_MBA0770828,GWHPACDB051680,KAE8695444,Hum_XP_021283093,Tca_XP_007030715
1858,Graim_XP_012492973,OMO62348,Corc_OMO98945,Dzib_XP_022742321,Ga11G0954,MBA0699224,Garm_MBA0833023,KAB2005777,Gdar_TYG47151,Ghir_XP_016721637,Glax_MBA0727895,Glob_MBA0574193,Gmus_TYI57716,Gsch_MBA0860290,Gtom_TYH46116,Gtri_MBA0783892,GWHPACDB052085,KAE8734700,Hum_XP_021282322,Tca_XP_007030718


In [5]:
#make a new dataframe 'newdf' to store geneids/nicknames
df = pd.read_csv("Family_gene_table.csv")
data = df.columns.values.tolist()
newdf = pd.DataFrame(data)
newdf.rename(columns={newdf.columns[0]: "Genome name"}, inplace=True)
newdf["nicknames"] = ""
newdf

,Genome name,nicknames
0,RefGeneID,
1,Ccapsularis,
2,Colitorius.fixed,
3,Dzibethinus.fixed,
4,Garboreum.fixed,
5,Garidum,
6,Garmourianum.fixed,
7,Gbarbadense,
8,Gdarwinii.fixed,
9,Ghirsutum.fixed,


In [6]:
#for each species column in the unique_species.csv, extract the first 4 characters.
#important note: if three characters is not enough to have a unique identifier for each species, increase to 5.

nickname_data = []
for column in df:
    df[column] = df[column].astype('string').str[0:4]
    df[column] = df[column].astype('category')
    nickname = df[column].cat.categories.values.tolist()
    nickname_data.append(nickname)
print(nickname_data)

[['Grai'], ['OMO4', 'OMO5', 'OMO6', 'OMO7', 'OMO8', 'OMO9', 'OMP0', 'OMP1'], ['Corc'], ['Dzib'], ['Ga01', 'Ga02', 'Ga03', 'Ga04', 'Ga05', 'Ga06', 'Ga07', 'Ga08', 'Ga09', 'Ga10', 'Ga11', 'Ga12', 'Ga13', 'Ga14'], ['MBA0'], ['Garm'], ['KAB1', 'KAB2'], ['Gdar'], ['Ghir'], ['Glax'], ['Glob'], ['Gmus'], ['Gsch'], ['Gtom'], ['Gtri'], ['GWHP'], ['KAE8'], ['Hum_'], ['Tca_']]


In [7]:
#check that no values are duplicated, all should be false
check_dup_df = pd.DataFrame(nickname_data)
check_dup_df.duplicated()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
dtype: bool

In [8]:
newdf["nicknames"] = nickname_data
newdf

,Genome name,nicknames
0,RefGeneID,[Grai]
1,Ccapsularis,"[OMO4, OMO5, OMO6, OMO7, OMO8, OMO9, OMP0, OMP1]"
2,Colitorius.fixed,[Corc]
3,Dzibethinus.fixed,[Dzib]
4,Garboreum.fixed,"[Ga01, Ga02, Ga03, Ga04, Ga05, Ga06, Ga07, Ga0..."
5,Garidum,[MBA0]
6,Garmourianum.fixed,[Garm]
7,Gbarbadense,"[KAB1, KAB2]"
8,Gdarwinii.fixed,[Gdar]
9,Ghirsutum.fixed,[Ghir]


In [9]:
newdf.to_csv("uniqueSpeciesNicknames.csv")

#delete header from csv file
!sed -i -e 1,1d uniqueSpeciesNicknames.csv

#check that the four letter geneid doesnt match anything but itself or its species in the final csv

In [10]:
#remove extra characters
!sed -i 's/"//g' uniqueSpeciesNicknames.csv
!sed -i 's/\[//g' uniqueSpeciesNicknames.csv
!sed -i 's/\]//g' uniqueSpeciesNicknames.csv
!sed -i "s/'//g" uniqueSpeciesNicknames.csv
!sed -i "s/ //g" uniqueSpeciesNicknames.csv

In [11]:
%%script bash
    cat uniqueSpeciesNicknames.csv | while read line;
    do
        #line="$(sed -n '10p' uniqueSpeciesNicknames.csv)"
        #echo "The line is: $line"
        mapfile -t my_array < <( for i in ${line//,/ }; do echo "$i"; done )
        #echo "The array is: ${my_array[@]}"
        echo "Genome species name is: ${my_array[1]}"
        length="${#my_array[@]}"
        echo "GeneID identifiers are:"
        for (( j=2; j<length; j++ )); do echo "${my_array[$j]}"; done
    done

Genome species name is: RefGeneID
GeneID identifiers are:
Grai
Genome species name is: Ccapsularis
GeneID identifiers are:
OMO4
OMO5
OMO6
OMO7
OMO8
OMO9
OMP0
OMP1
Genome species name is: Colitorius.fixed
GeneID identifiers are:
Corc
Genome species name is: Dzibethinus.fixed
GeneID identifiers are:
Dzib
Genome species name is: Garboreum.fixed
GeneID identifiers are:
Ga01
Ga02
Ga03
Ga04
Ga05
Ga06
Ga07
Ga08
Ga09
Ga10
Ga11
Ga12
Ga13
Ga14
Genome species name is: Garidum
GeneID identifiers are:
MBA0
Genome species name is: Garmourianum.fixed
GeneID identifiers are:
Garm
Genome species name is: Gbarbadense
GeneID identifiers are:
KAB1
KAB2
Genome species name is: Gdarwinii.fixed
GeneID identifiers are:
Gdar
Genome species name is: Ghirsutum.fixed
GeneID identifiers are:
Ghir
Genome species name is: Glaxum.fixed
GeneID identifiers are:
Glax
Genome species name is: Globatum.fixed
GeneID identifiers are:
Glob
Genome species name is: Gmustelinum.fixed
GeneID identifiers are:
Gmus
Genome species n

### When MAFFT is finished, replace gene ids

In [66]:
#check that it worked (should see .aln files)
os.chdir("LociForAlignment")
!ls

locus_0.txt.fasta	  locus_1556.txt.fasta	    locus_440.txt.fasta
locus_0.txt.fasta.aln	  locus_1556.txt.fasta.aln  locus_440.txt.fasta.aln
locus_1.txt.fasta	  locus_1557.txt.fasta	    locus_441.txt.fasta
locus_1.txt.fasta.aln	  locus_1557.txt.fasta.aln  locus_441.txt.fasta.aln
locus_10.txt.fasta	  locus_1558.txt.fasta	    locus_442.txt.fasta
locus_10.txt.fasta.aln	  locus_1558.txt.fasta.aln  locus_442.txt.fasta.aln
locus_100.txt.fasta	  locus_1559.txt.fasta	    locus_443.txt.fasta
locus_100.txt.fasta.aln   locus_1559.txt.fasta.aln  locus_443.txt.fasta.aln
locus_1000.txt.fasta	  locus_156.txt.fasta	    locus_444.txt.fasta
locus_1000.txt.fasta.aln  locus_156.txt.fasta.aln   locus_444.txt.fasta.aln
locus_1001.txt.fasta	  locus_1560.txt.fasta	    locus_445.txt.fasta
locus_1001.txt.fasta.aln  locus_1560.txt.fasta.aln  locus_445.txt.fasta.aln
locus_1002.txt.fasta	  locus_1561.txt.fasta	    locus_446.txt.fasta
locus_1002.txt.fasta.aln  locus_1561.txt.fasta.aln  locus_446.txt.fasta.aln
locus_1

In [67]:
#check .aln file
!cat locus_3.txt.fasta.aln

>OMO86276.1 hypothetical protein CCACVL1_09673 [Corchorus capsularis]
---------------------MEEELYRYRLDD-------DINVRVDNFSA----AGVTN
DFFS-PVD-YNLLQFDHQKHDHHTDVSGSH-MSHLLKTQIASHPRYPNLVSAFIECQKVG
APPELASLLEEIGREN-LPIRGCSEMGADPELDEFMESYCEVLHRYKEELSKPFD--EAT
TFLSNIESQLNNLCKLGQLSNSLDY--------RSDEAAGTSEEELSGGE-VETSESQES
AGGGGSQLSQDIKGMLMQKYSGY-------------------------LSSLRKEFLKKR
KKGKLPKDARMALLQWWNNHYRWPY-PT--------------------------------
-EEE---------------KLK--LSEITGLDQKQINNWFINQRKRHWKPSEDMKFALLE
GVAANI-GGPAMFLDTGLGTASDDI-----------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
------------------------------------------------------------
---------------

In [68]:
#make a new Test directory to modify species names
os.mkdir('../Test')
#copy .aln files over
!cp *.aln ../Test/

In [13]:
os.chdir('Test')
!ls

locus_0.txt.fasta.aln	  locus_1556.txt.fasta.aln  locus_440.txt.fasta.aln
locus_1.txt.fasta.aln	  locus_1557.txt.fasta.aln  locus_441.txt.fasta.aln
locus_10.txt.fasta.aln	  locus_1558.txt.fasta.aln  locus_442.txt.fasta.aln
locus_100.txt.fasta.aln   locus_1559.txt.fasta.aln  locus_443.txt.fasta.aln
locus_1000.txt.fasta.aln  locus_156.txt.fasta.aln   locus_444.txt.fasta.aln
locus_1001.txt.fasta.aln  locus_1560.txt.fasta.aln  locus_445.txt.fasta.aln
locus_1002.txt.fasta.aln  locus_1561.txt.fasta.aln  locus_446.txt.fasta.aln
locus_1003.txt.fasta.aln  locus_1562.txt.fasta.aln  locus_447.txt.fasta.aln
locus_1004.txt.fasta.aln  locus_1563.txt.fasta.aln  locus_448.txt.fasta.aln
locus_1005.txt.fasta.aln  locus_1564.txt.fasta.aln  locus_449.txt.fasta.aln
locus_1006.txt.fasta.aln  locus_1565.txt.fasta.aln  locus_45.txt.fasta.aln
locus_1007.txt.fasta.aln  locus_1566.txt.fasta.aln  locus_450.txt.fasta.aln
locus_1008.txt.fasta.aln  locus_1567.txt.fasta.aln  locus_451.txt.fasta.aln
locus_1009.txt.fas

In [14]:
#make sure you are in the right directory!
!cat ../uniqueSpeciesNicknames.csv

0,RefGeneID,Grai
1,Ccapsularis,OMO4,OMO5,OMO6,OMO7,OMO8,OMO9,OMP0,OMP1
2,Colitorius.fixed,Corc
3,Dzibethinus.fixed,Dzib
4,Garboreum.fixed,Ga01,Ga02,Ga03,Ga04,Ga05,Ga06,Ga07,Ga08,Ga09,Ga10,Ga11,Ga12,Ga13,Ga14
5,Garidum,MBA0
6,Garmourianum.fixed,Garm
7,Gbarbadense,KAB1,KAB2
8,Gdarwinii.fixed,Gdar
9,Ghirsutum.fixed,Ghir
10,Glaxum.fixed,Glax
11,Globatum.fixed,Glob
12,Gmustelinum.fixed,Gmus
13,Gschwendimanii.fixed,Gsch
14,Gtomentosum.fixed,Gtom
15,Gtrilobum.fixed,Gtri
16,Hcannabinus,GWHP
17,Hsyriacus,KAE8
18,Humbratica.fixed,Hum_
19,Tcacao.fixed,Tca_


In [99]:
#replace all geneids with species names in the Test folder
!ls

locus_0.txt.fasta.aln	  locus_1556.txt.fasta.aln  locus_440.txt.fasta.aln
locus_1.txt.fasta.aln	  locus_1557.txt.fasta.aln  locus_441.txt.fasta.aln
locus_10.txt.fasta.aln	  locus_1558.txt.fasta.aln  locus_442.txt.fasta.aln
locus_100.txt.fasta.aln   locus_1559.txt.fasta.aln  locus_443.txt.fasta.aln
locus_1000.txt.fasta.aln  locus_156.txt.fasta.aln   locus_444.txt.fasta.aln
locus_1001.txt.fasta.aln  locus_1560.txt.fasta.aln  locus_445.txt.fasta.aln
locus_1002.txt.fasta.aln  locus_1561.txt.fasta.aln  locus_446.txt.fasta.aln
locus_1003.txt.fasta.aln  locus_1562.txt.fasta.aln  locus_447.txt.fasta.aln
locus_1004.txt.fasta.aln  locus_1563.txt.fasta.aln  locus_448.txt.fasta.aln
locus_1005.txt.fasta.aln  locus_1564.txt.fasta.aln  locus_449.txt.fasta.aln
locus_1006.txt.fasta.aln  locus_1565.txt.fasta.aln  locus_45.txt.fasta.aln
locus_1007.txt.fasta.aln  locus_1566.txt.fasta.aln  locus_450.txt.fasta.aln
locus_1008.txt.fasta.aln  locus_1567.txt.fasta.aln  locus_451.txt.fasta.aln
locus_1009.txt.fas

In [ ]:
#%%script bash
#    line="$(sed -n '4p' ../uniqueSpeciesNicknames.csv)"
#    echo "$line"
#    mapfile -t my_array < <( for i in ${line//,/ }; do echo "$i"; done )
#    geneid="${my_array[2]}"
#    name="${my_array[1]}"
#    echo "Replacing $geneid with $name"
#    sed -i "s/>$geneid.*/>$name/g" locus_0.txt.fasta.aln

In [ ]:
#replace gene names with Species names in Test folder

In [15]:
%%script bash
    cat ../uniqueSpeciesNicknames.csv | while read line;
    do
        mapfile -t my_array < <( for i in ${line//,/ }; do echo "$i"; done )
        length="${#my_array[@]}"
        for (( j=2; j<length; j++ ))
        do
            geneid="${my_array[j]}"
            name="${my_array[1]}"
            echo "Replacing $geneid with $name"
            for file in *; do sed -i "s/>$geneid.*/>$name/g" $file; done
        done
    done


Replacing Grai with RefGeneID
Replacing OMO4 with Ccapsularis
Replacing OMO5 with Ccapsularis
Replacing OMO6 with Ccapsularis
Replacing OMO7 with Ccapsularis
Replacing OMO8 with Ccapsularis
Replacing OMO9 with Ccapsularis
Replacing OMP0 with Ccapsularis
Replacing OMP1 with Ccapsularis
Replacing Corc with Colitorius.fixed
Replacing Dzib with Dzibethinus.fixed
Replacing Ga01 with Garboreum.fixed
Replacing Ga02 with Garboreum.fixed
Replacing Ga03 with Garboreum.fixed
Replacing Ga04 with Garboreum.fixed
Replacing Ga05 with Garboreum.fixed
Replacing Ga06 with Garboreum.fixed
Replacing Ga07 with Garboreum.fixed
Replacing Ga08 with Garboreum.fixed
Replacing Ga09 with Garboreum.fixed
Replacing Ga10 with Garboreum.fixed
Replacing Ga11 with Garboreum.fixed
Replacing Ga12 with Garboreum.fixed
Replacing Ga13 with Garboreum.fixed
Replacing Ga14 with Garboreum.fixed
Replacing MBA0 with Garidum
Replacing Garm with Garmourianum.fixed
Replacing KAB1 with Gbarbadense
Replacing KAB2 with Gbarbadense
Repl

In [ ]:
#check that each species appears once per file, no output is good

In [16]:
%%script bash
    cat ../uniqueSpeciesNicknames.csv | while read line;
    do
        mapfile -t my_array < <( for i in ${line//,/ }; do echo "$i"; done )
        for file in *
        do
            count=$(grep -o "${my_array[1]}" $file | wc -l)
            if [ $count != 1 ]
            then
                echo $file; echo ${my_array[1]}; echo $count
            fi
        done
    done

In [93]:
os.chdir('../Test')
!ls

Shuffle_Set300		  locus_1555.txt.fasta.aln  locus_440.txt.fasta.aln
Shuffle_Set600		  locus_1556.txt.fasta.aln  locus_441.txt.fasta.aln
locus_0.txt.fasta.aln	  locus_1557.txt.fasta.aln  locus_442.txt.fasta.aln
locus_1.txt.fasta.aln	  locus_1558.txt.fasta.aln  locus_443.txt.fasta.aln
locus_10.txt.fasta.aln	  locus_1559.txt.fasta.aln  locus_444.txt.fasta.aln
locus_100.txt.fasta.aln   locus_156.txt.fasta.aln   locus_445.txt.fasta.aln
locus_1000.txt.fasta.aln  locus_1560.txt.fasta.aln  locus_446.txt.fasta.aln
locus_1001.txt.fasta.aln  locus_1561.txt.fasta.aln  locus_447.txt.fasta.aln
locus_1002.txt.fasta.aln  locus_1562.txt.fasta.aln  locus_448.txt.fasta.aln
locus_1003.txt.fasta.aln  locus_1563.txt.fasta.aln  locus_449.txt.fasta.aln
locus_1004.txt.fasta.aln  locus_1564.txt.fasta.aln  locus_45.txt.fasta.aln
locus_1005.txt.fasta.aln  locus_1565.txt.fasta.aln  locus_450.txt.fasta.aln
locus_1006.txt.fasta.aln  locus_1566.txt.fasta.aln  locus_451.txt.fasta.aln
locus_1007.txt.fasta.aln  locus_15

In [76]:
##replace * and . characters in fasta files
!for file in *; do sed -i 's/\.//g' $file; done
!for file in *; do sed -i 's/\*//g' $file; done

In [101]:
#check that it worked
!cat locus_1.txt.fasta.aln

>Ccapsularis
------------------------------------------------------------
------MDAESWSARLSSASKRY-QSALQSRSDMFMGFEEIDGEDDIREEFPCPFCSEYF
DIVGLCCHIDDEHPVEAKNG----------------------------RKRKSRKGGSHS
TLSLLRKELREGNLQSLFGGSSCIMSSSNSAPDPLLSSFILPM-VDDFVSVQPHFSSETS
TTKKSADVNKSERNVQAS-PLSVKDQEEKAKRCEFVQGLLLSTILDDSL
>Colitorius.fixed
------------------------------------------------------------
------MDAESWSARLSSASKRY-QSALQSRSDMFMGFEEIDGEDDIREEFPCPFCSEYF
DIVGLCCHIDDEHPVEAKNGVCPVCAMRVGVDM---------------RKRKSRKGGSHS
TLSLLRKELREGNLQSLFGGSSCIMSSSNSAPDPLLSSFILPM-VDDFVSVQPHFSSETS
TTKKSADVNKSERNVQASPPLSVKDQEEKAKRCEFVQGLLLSTILDDSL
>Dzibethinus.fixed
------------------------------------------------------------
------MDADSWSARLSSSSQRY-QSALQSRTDMFMAFEEIDGEDNIREEFPCPFCSEYF
DIVGLCCHIDDEHPVEAKNGVCPVCSMRVGVDMVAHITLQHGNIFKMQRKRKSHKGASHS
ILSLLKKELREGNLQSLFGGSSYIVSSSNSAPDPLLSSFILPI-VDDFVSVQPPFSSETG
TTKRSSNVNKSERNVESS-PLAVKDQEEKAKRCEFVQRLLLSTVLDDIL
>Garmourianum.fixed
------------------------------------------------

## (Skip this step because raxml-ng can use fasta files as input) STEP 4: Convert fasta alignments to phylip 

In [ ]:
!ls

In [ ]:
#copy FastatoPylip.pl to this directory
os.chdir('../../')
!cp FastatoPhylip.pl Malvaceae/Test/

In [ ]:
os.chdir('Malvaceae/Test')
!ls

In [ ]:
#convert all files to Phylip
!for g in *.aln; do perl FastatoPhylip.pl $g > $g.phy; done

In [ ]:
#check that it worked
!ls

In [ ]:
#check that it worked
!cat locus_1.txt.fasta.aln.phy

In [ ]:
#move all Phylip files to new directory
os.mkdir("../PhylipAlignments")
!mv *.phy ../PhylipAlignments

In [ ]:
#os.chdir("../PhylipAlignments")
#!ls

In [ ]:
#copy catfasta2phyml.pl
import os
#os.chdir('../../')
#!cp catfasta2phyml.pl Malvaceae/Test/

## STEP 5: Concatenate all good loci, run RAxML analysis


In [82]:
#make sure you are in the Test directory
os.chdir ("../")
os.chdir("Test")
!ls

locus_0.txt.fasta.aln	  locus_1556.txt.fasta.aln  locus_440.txt.fasta.aln
locus_1.txt.fasta.aln	  locus_1557.txt.fasta.aln  locus_441.txt.fasta.aln
locus_10.txt.fasta.aln	  locus_1558.txt.fasta.aln  locus_442.txt.fasta.aln
locus_100.txt.fasta.aln   locus_1559.txt.fasta.aln  locus_443.txt.fasta.aln
locus_1000.txt.fasta.aln  locus_156.txt.fasta.aln   locus_444.txt.fasta.aln
locus_1001.txt.fasta.aln  locus_1560.txt.fasta.aln  locus_445.txt.fasta.aln
locus_1002.txt.fasta.aln  locus_1561.txt.fasta.aln  locus_446.txt.fasta.aln
locus_1003.txt.fasta.aln  locus_1562.txt.fasta.aln  locus_447.txt.fasta.aln
locus_1004.txt.fasta.aln  locus_1563.txt.fasta.aln  locus_448.txt.fasta.aln
locus_1005.txt.fasta.aln  locus_1564.txt.fasta.aln  locus_449.txt.fasta.aln
locus_1006.txt.fasta.aln  locus_1565.txt.fasta.aln  locus_45.txt.fasta.aln
locus_1007.txt.fasta.aln  locus_1566.txt.fasta.aln  locus_450.txt.fasta.aln
locus_1008.txt.fasta.aln  locus_1567.txt.fasta.aln  locus_451.txt.fasta.aln
locus_1009.txt.fas

In [17]:
#Shuffle sets of 100, 200, and 300 random Loci and store in new folders. 
#This is because depending on dataset we might have to use less loci than 200, or we might be able to use more.
#os.mkdir("Shuffle_Set100")
os.mkdir("Shuffle_Set600")
os.mkdir("Shuffle_Set300")
#!cp $(ls *.aln | shuf -n 100) Shuffle_Set100
!cp $(ls *.aln | shuf -n 600) Shuffle_Set600
!cp $(ls *.aln | shuf -n 300) Shuffle_Set300

In [ ]:
#Or just Shuffle all if you dont have a lot of loci
os.mkdir("Shuffle_Set_all")
!cp *.aln Shuffle_Set_all

In [18]:
#Concatenate shuffle sets with catfasta2phyml.pl
#!perl ../../catfasta2phyml.pl Shuffle_Set100/* > Shuffle_Set100.fa
!perl ../../catfasta2phyml.pl Shuffle_Set600/* > Shuffle_Set600.fa
!perl ../../catfasta2phyml.pl Shuffle_Set300/* > Shuffle_Set300.fa

Shuffle_Set600/locus_1000.txt.fasta.aln = 1-1736
Shuffle_Set600/locus_1002.txt.fasta.aln = 1737-2683
Shuffle_Set600/locus_1009.txt.fasta.aln = 2684-3522
Shuffle_Set600/locus_101.txt.fasta.aln = 3523-5153
Shuffle_Set600/locus_1011.txt.fasta.aln = 5154-5519
Shuffle_Set600/locus_1012.txt.fasta.aln = 5520-6027
Shuffle_Set600/locus_1014.txt.fasta.aln = 6028-6310
Shuffle_Set600/locus_1015.txt.fasta.aln = 6311-6577
Shuffle_Set600/locus_1016.txt.fasta.aln = 6578-7268
Shuffle_Set600/locus_1017.txt.fasta.aln = 7269-7847
Shuffle_Set600/locus_1019.txt.fasta.aln = 7848-8404
Shuffle_Set600/locus_1021.txt.fasta.aln = 8405-9664
Shuffle_Set600/locus_1022.txt.fasta.aln = 9665-10225
Shuffle_Set600/locus_1025.txt.fasta.aln = 10226-11750
Shuffle_Set600/locus_1032.txt.fasta.aln = 11751-12279
Shuffle_Set600/locus_1033.txt.fasta.aln = 12280-12495
Shuffle_Set600/locus_1035.txt.fasta.aln = 12496-13268
Shuffle_Set600/locus_1038.txt.fasta.aln = 13269-14313
Shuffle_Set600/locus_1039.txt.fasta.aln = 14314-14996
Shu

Shuffle_Set600/locus_548.txt.fasta.aln = 310032-310440
Shuffle_Set600/locus_549.txt.fasta.aln = 310441-311153
Shuffle_Set600/locus_552.txt.fasta.aln = 311154-311795
Shuffle_Set600/locus_553.txt.fasta.aln = 311796-313113
Shuffle_Set600/locus_557.txt.fasta.aln = 313114-313897
Shuffle_Set600/locus_558.txt.fasta.aln = 313898-314090
Shuffle_Set600/locus_56.txt.fasta.aln = 314091-315274
Shuffle_Set600/locus_560.txt.fasta.aln = 315275-315905
Shuffle_Set600/locus_561.txt.fasta.aln = 315906-316279
Shuffle_Set600/locus_567.txt.fasta.aln = 316280-317154
Shuffle_Set600/locus_571.txt.fasta.aln = 317155-318243
Shuffle_Set600/locus_572.txt.fasta.aln = 318244-318770
Shuffle_Set600/locus_58.txt.fasta.aln = 318771-319005
Shuffle_Set600/locus_583.txt.fasta.aln = 319006-319950
Shuffle_Set600/locus_587.txt.fasta.aln = 319951-320420
Shuffle_Set600/locus_588.txt.fasta.aln = 320421-320690
Shuffle_Set600/locus_595.txt.fasta.aln = 320691-321573
Shuffle_Set600/locus_596.txt.fasta.aln = 321574-321892
Shuffle_Set6

Shuffle_Set300/locus_0.txt.fasta.aln = 1-174
Shuffle_Set300/locus_1002.txt.fasta.aln = 175-1121
Shuffle_Set300/locus_1009.txt.fasta.aln = 1122-1960
Shuffle_Set300/locus_1011.txt.fasta.aln = 1961-2326
Shuffle_Set300/locus_1018.txt.fasta.aln = 2327-2774
Shuffle_Set300/locus_1021.txt.fasta.aln = 2775-4034
Shuffle_Set300/locus_1028.txt.fasta.aln = 4035-4557
Shuffle_Set300/locus_1035.txt.fasta.aln = 4558-5330
Shuffle_Set300/locus_1037.txt.fasta.aln = 5331-6511
Shuffle_Set300/locus_1039.txt.fasta.aln = 6512-7194
Shuffle_Set300/locus_1046.txt.fasta.aln = 7195-7494
Shuffle_Set300/locus_1058.txt.fasta.aln = 7495-8373
Shuffle_Set300/locus_1071.txt.fasta.aln = 8374-9823
Shuffle_Set300/locus_1073.txt.fasta.aln = 9824-10592
Shuffle_Set300/locus_1074.txt.fasta.aln = 10593-11164
Shuffle_Set300/locus_1089.txt.fasta.aln = 11165-11645
Shuffle_Set300/locus_11.txt.fasta.aln = 11646-11861
Shuffle_Set300/locus_1103.txt.fasta.aln = 11862-12691
Shuffle_Set300/locus_1114.txt.fasta.aln = 12692-13013
Shuffle_Set

Shuffle_Set300/locus_85.txt.fasta.aln = 186531-188711
Shuffle_Set300/locus_857.txt.fasta.aln = 188712-189077
Shuffle_Set300/locus_862.txt.fasta.aln = 189078-190443
Shuffle_Set300/locus_867.txt.fasta.aln = 190444-191141
Shuffle_Set300/locus_873.txt.fasta.aln = 191142-191864
Shuffle_Set300/locus_877.txt.fasta.aln = 191865-192855
Shuffle_Set300/locus_884.txt.fasta.aln = 192856-193481
Shuffle_Set300/locus_890.txt.fasta.aln = 193482-194220
Shuffle_Set300/locus_896.txt.fasta.aln = 194221-194344
Shuffle_Set300/locus_905.txt.fasta.aln = 194345-195859
Shuffle_Set300/locus_907.txt.fasta.aln = 195860-196321
Shuffle_Set300/locus_908.txt.fasta.aln = 196322-196697
Shuffle_Set300/locus_93.txt.fasta.aln = 196698-197011
Shuffle_Set300/locus_931.txt.fasta.aln = 197012-197197
Shuffle_Set300/locus_934.txt.fasta.aln = 197198-198043
Shuffle_Set300/locus_938.txt.fasta.aln = 198044-198424
Shuffle_Set300/locus_944.txt.fasta.aln = 198425-198742
Shuffle_Set300/locus_950.txt.fasta.aln = 198743-201724
Shuffle_Set3

In [ ]:
!perl ../../catfasta2phyml.pl Shuffle_Set_all/* > Shuffle_Set_all.fa

In [19]:
#check ShuffleSet files
!less Shuffle_Set300.fa

20 206671[?1h=
Ccapsularis            --------------------------------------MGLLDQLWDDTVAGPRPDNGLGKLRKHSTFTFRPNSSAKESDGGTVRSYGDETS-EEATRVTRSIMIVKPPGYQSGSPPVSPAGSTPP----VSPFSGGRETFRFRRRSTSDAYE----KASEVGPRSPR-PPYDV
Colitorius.fixed       --------------------------------------MGLLDQLWDDTVAGPRPDNGLGKLRKHSTFTFRPNSSAKESDGGSVRSYGDETS-EEATRVTRSIMIVKPPGYQSGSPPVSPAGSTPP----VSPFSGGRETFRFRRRSTSDVYE----KASEVGPRSPR-PPYDV
Dzibethinus.fixed      --------------------------------------MGLLDQLWDDTVAGPRPDNGLGKLRKHSTFTFRPN-AAKESDGGSVRSYGDQTS-EEATRVTRTIMIVKPPGYQSGSPPVSPAGSTPP----VSPFSGGRDLYRFRRRSTSDAYE----KASEVGPRSPR-PPYDV
Garboreum.fixed        --------------------------------------MGLLDQLWDDTVAGPRPDNGLGKLRKHSTFTFRPN-SAKESDGGSVRSYGDEAP-EEATRVTRSIMIVKPPGYQSGSPPVSPAGSTPP----VSPFSGGRESYRFRRRSTSDAFE----KANEVGPRSPR-PPYDV
Garidum                --------------------------------------MGLLDQLWDDTVAGPRPDTGLGKLRKQSKFTFWPN-SARESDGGSVISNGDETSLEKATRVTRTIMIVKPPGYQTGSPPVSPAGTTPP----VSPFAGSKESHQFRRRSISDAYE----KAGEVGARSPH-

In [20]:
#copy ShuffleSets to Final Concatenated folder and run raxml. I could only get this to work with raxml-ng. Details: https://isu-molphyl.github.io/EEOB563/computer_labs/lab4/raxml-ng.html
#os.mkdir("../FinalConcatenated")
#!cp Shuffle_Set_all.fa ../FinalConcatenated
#!cp Shuffle_Set100.fa ../FinalConcatenated
!cp Shuffle_Set600.fa ../FinalConcatenated
!cp Shuffle_Set300.fa ../FinalConcatenated

In [21]:
os.chdir("../FinalConcatenated")
!ls

Attempt_1  Shuffle_Set300.fa  Shuffle_Set600.fa  parse300.raxml.log


In [22]:
#check that the files were copied
!head Shuffle_Set600.fa

20 440683
Ccapsularis            ------------------------------------MSNTVGNNLIQQRFLRPTVLEHQSKLKNSS---TNSLCATASLNQSSTLPRKFQISTKFYGDSLSKRKQKFAMGSQRA-VSFTPQAVLAADPASEQLGKFNVDGNIELQVDAS-VPTSGAITQINFRIMYTSDSLLLHWGGISDRNEKWILPSRQPEGTRNHKNRALRTPFVKSGSSSYLKLEIDDPRIQAIEFLIFDEARNK-------------------W----------IKNNGQNFLVKLPRRETSISNISVPEDLVQVQAYLRWERKGKQMYTPEKEK---------EEYEAARAELLEEIARGASVDDIRARLTKKDGQEYKETTINEPKNKIPDDLVQIQAYIRWEKAGKPNYSPDQQLGEFEEARKELQSELEKGVTLDEIRKKITKGEIQTKVSKQLQHKRYFSAERIQRKKRDLMQLLNKHVVKPVEESIYSEPEPKALTAVELFAKEKELDGSTVLNKKKYKLGNGELMVLVTKPAGKTRIQLATDFEEPLTLHWALSKEPGEWLPPPPSVLPPGSVSLG------------GAAESQFSISTYADLPK-------QVQCLEIEIEDSNFKG-------MPFVLLSGGNWIKNNGSDFYVEFSQKVKEVQ------KDAGDGKGTSKGLLDRIADLESEAQKSFMHRFNIASDLMDQAKDTGELGLAGILVWMRFMATRQLIWNKNYNVKPR-----------------------------EISKAQDRLTDLLQ----SIYTTQPQYRELLRMIMSTVGRGGEGDVGQRIRDEILVIQ------RNNDCKGGMMEEWHQKLHNNTSPDDVV-----IC-QALIDYIKSDFDISVYWKTLNENGITKERLLSYDRAIHSEPNFRRDQKDGLLRDLGHYMRTLKAVHSGADLESAISNCMGYTAEGQGFMVGVQINPIAG

### Running & optimizing large alignments with raxml-ng
#### Step 1: Load raxml-ng from terminal with command 
`module load raxml-ng/0.9.0`
#### Step 2: Run raxml-ng in 'parse' mode first. Submit the following job in the terminal with your .fa or .phy file

`bsub -q interactive -Is -W 1:00 -n 1 -R "span[hosts=1]" singularity exec $RAXMLNGIMG raxml-ng-mpi --msa Shuffle_Set200.fa --model JTT+G --parse --prefix parse200`

* This will create a log file that you can check to see the optimized number of threads. It also has information about memory requirements.  
* You can run parse on different numbers of concatenated loci (the different shuffle sets) to see what is best time-wise. You can possibly use more than 300 loci if there is a high percentage of invariant sites and a low number of species.
* --model JTT+G specifies amino acid alignment. More detail [here](https://github.com/amkozlov/raxml-ng/wiki/Input-data).

#### Step 3: Run raxml-ng with updated thread number

* Once you know how many threads you want to use, change the numbers in the command below at '-n X' and '--threads X', where X equals your number of desired threads. I double checked with the ghpcc system admin people and this is the most efficient way to run the program. More detail [here](https://github.com/amkozlov/raxml-ng/wiki/Parallelization#so-how-many-threadscores-should-i-use).

`bsub -q long -W 8:00 -n 32 -o "raxml_job.out" -e "raxml_job.err" -R "span[hosts=1]" -R "rusage[mem=2000]" singularity exec $RAXMLNGIMG raxml-ng-mpi --msa Shuffle_Set200.fa --model JTT+G --threads 32`

#### Example runtimes:

* For 200 concatenated Loci of Rutaceae (w/ 10 species), there were 92% invariant sites, 80MB for the estimated memory requirements and 11 threads recommended. I ran this on 1 thread with 2000MB and it only took 15 mins to complete. 
* For 100 concatenated Loci of Malvaceae (w/ 19 species), there were 60% invariant sites, 1000MB estimated memory and 39 threads recommended. This took 3 hours to complete on 32 threads. I accidently didn't run this on the reduced file, it might have been faster if I had done that.


## Step 6: Looking at generated ML tree

In [37]:
#install toytree to look at trees in notebook
!pip install toytree

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg/python3/3.6_intel_pu3_2018.3.039/intelpython3/bin/python -m pip install --upgrade pip' command.


In [26]:
os.chdir("../../Cucurbitaceae/FinalConcatenated")
!ls

Cucurbitaceae_ConstraintTree.pdf	 Shuffle_Set300.fa.raxml.startTree
Cucurbitaceae_ConstraintTree.phy	 parse100.raxml.log
Cucurbitaceae_ConstraintTree_Thomas.phy  parse100.raxml.rba
Shuffle_Set100.fa			 parse200.raxml.log
Shuffle_Set200.fa			 parse200.raxml.rba
Shuffle_Set300.fa			 parse200.raxml.reduced.phy
Shuffle_Set300.fa.raxml.bestModel	 parse300.raxml.log
Shuffle_Set300.fa.raxml.bestTree	 parse300.raxml.rba
Shuffle_Set300.fa.raxml.log		 parse300.raxml.reduced.phy
Shuffle_Set300.fa.raxml.mlTrees		 raxml_job_300.err
Shuffle_Set300.fa.raxml.rba		 raxml_job_300.out
Shuffle_Set300.fa.raxml.reduced.phy


In [65]:
import sys
#tell python to look in your local folder for the newly installed packages
sys.path.append('/home/ad32a/.local/lib/python3.6/site-packages')

In [4]:
#import the new packages to python
import toytree
import toyplot
import numpy as np

In [31]:
#load in the new tree
tre = toytree.tree("Shuffle_Set300.fa.raxml.bestTree")
#load in the new tree


In [73]:
#draw tree
tre.unroot().write()

'(Dryas_drummondii:1,(Rosa_chinensis:1,(Potentilla_micrantha:1,(Fragaria_iinumae:1,Fragaria_nilgerrensis:1)0:1)0:1)0:1,((Eriobotrya_japonica:1,(Pyrus_betulifolia:1,(Malus_domestica:1,Malus_baccata:1)0:1)0:1)0:1,(Prunus_avium:1,((Prunus_salicina:1,(Prunus_mume:1,Prunus_armeniaca:1)0:1)0:1,(Prunus_ferganensis:1,Prunus_persica:1,Prunus_dulcis:1)0:1)0:1)0:1)0:1);'

In [ ]:
!cat Malvaceae_Shuffle_Set3.fa.raxml.bestTree

In [28]:
tre.draw(tip_labels_align=True, node_labels='support');

<svg class="toyplot-canvas-Canvas" height="275.0px" id="taa724a798ac44230acb4e54e07c16b1c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 315.0 275.0" width="315.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0 0 0 0 0 0 0 Cargyrosperma Cmoschata Cpepo RefGeneID Sgrosvenorii Mcharantia Clanatus Lsiceraria Csativus Cmelo Bhispida

In [ ]:
tre3 = toytree.tree("(((((Cor,Omo),(Tca,Hum)),Dzi),(Kae,Gwh)),((Gra,Ghi,Gda,Gla,Gto,Mba),(Gmu,Glo,Gtr,Gsc,Kab,Gar)));")
tre3.draw(tip_labels_align=True);

In [ ]:
text_file = open("constraint.txt", "w")
n = text_file.write("(((((Cor,Omo),(Tca,Hum)),Dzi),(Kae,Gwh)),((Gra,Ghi,Gda,Gla,Gto,Mba),(Gmu,Glo,Gtr,Gsc,Kab,Gar)));")
text_file.close()

In [ ]:
tre4 = toytree.tree("constraint.txt")
tre4.draw(tip_labels_align=True);

In [ ]:
label_list = "1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19"
tre.draw(tip_labels=label_list)

In [ ]:
merged_df

In [ ]:
label_list3 = "Gossypium armourianum","Gossypium barbadense","Gossypium tomentosum","Gossypium darwinii","Gossypium lobatum","Gossypium mustelinum","Gossypium schwendimanii", "Gossypium arboreum", "Gossypium laxum", "Gossypium hirsutum","Gossypium trilobum", "Gossypium aridum" , "Hibiscus cannabinus", "Hibiscus syriacus", "Durio zibethinus", "Corchorus capsularis", "Corchorus olitorius","Herrania umbratica","Theobroma cacao" 

In [63]:
tre.draw(tip_labels=True)

(<toyplot.canvas.Canvas at 0x2b9d4e45e2e8>,
 <toytree.Render.ToytreeMark at 0x2b9d4e583f28>)

<svg class="toyplot-canvas-Canvas" height="396.0px" id="tf753fba0185a4cd2ae66f0c1e950112c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 320.0 396.0" width="320.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Plunatus Pacutifolius Pvulgaris Vangularis Vradiata Vunguiculata Gsoja Gmax Ccajan Ssuberectus Aprecatorius Ljaponicus Trifolium RefGeneID Psativum Carietinum Ahypogaea Aduranensis Aipaensis Langustifolius Lalbus Stora

In [ ]:
tre.draw(tip_labels=True)
tre = tre.root(wildcard="Tca") #root the tree
tre.draw(tip_labels=label_list3)


In [32]:
canvas, axes, mark = tre.draw(width=700, height=500, tip_labels=True, tip_labels_align=True,use_edge_lengths=False)

<svg class="toyplot-canvas-Canvas" height="500.0px" id="td7d2810f05e94c0dbd4b9b9b01747859" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 700.0 500.0" width="700.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Cargyrosperma Cmoschata Cpepo Cmaxima Sgrosvenorii Mcharantia Clanatus Lsiceraria Csativus Cmelo Bhispida

In [ ]:
label_list3 = "Herrania umbratica","Theobroma cacao","Corchorus capsularis","Corchorus olitorius","Durio zibethinus","Hibiscus cannabinus","Hibiscus syriacus","Gossypium aridum","Gossypium armourianum","Gossypium barbadense","Gossypium tomentosum","Gossypium darwinii","Gossypium lobatum","Gossypium mustelinum","Gossypium schwendimanii","Gossypium trilobum","Gossypium arboreum","Gossypium laxum","Gossypium hirsutum"

In [33]:
import toyplot.pdf
toyplot.pdf.render(canvas, "Cucurbitaceae_Best_ML_tree.pdf")

In [ ]:
tre3 = toytree.tree("FINALTREE2.raxml.bestTree")
tre3.draw(tip_labels=True)
canvas, axes, mark = tre3.draw(width=1400, height=500, tip_labels=True)

In [ ]:
 bsub -q long -W 24:00 -n 32 -o "raxml_job_300_bs.out" -e "raxml_job_300_bs.err" -R "span[hosts=1]" -R "rusage[mem=2000]" singularity exec $RAXMLNGIMG raxml-ng-mpi --all --msa Shuffle_Set300.fa --model JTT+G --constraint-tree Cucurbitaceae_ConstraintTree.phy --threads 32 --bs-metric fbp,tbe


In [25]:
pip --version

pip 21.0.1 from /share/pkg/python3/3.6_intel_pu3_2018.3.039/intelpython3/lib/python3.6/site-packages/pip (python 3.6)
Note: you may need to restart the kernel to use updated packages.


## Step 7: Make allll the trees

In [ ]:
# navigate to the folder with all your annotated phylip files for each loci

In [ ]:
os.chdir("PhylipAlignments")
!ls

In [ ]:
#look at one to check it is right
!cat locus_1008.txt.fasta.aln.phy

#### Parse all alignment files and then run raxml on each parsed file

`bsub < raxml_all_parse.sh`

wait until completed (should only take a few mins)  

`bsub < raxml_all.sh`

#### Contents of raxml_all_parse.sh

`#!/bin/bash`    

`#BSUB -n 1
#BSUB -R rusage[mem=2048]
#BSUB -W 1:00
#BSUB -q short
#BSUB -R "span[hosts=1]"
#BSUB -o "raxml_all_job_parse.out"
#BSUB -e "raxml_all_job_parse.err"`

`module load raxml-ng/0.9.0
module load singularity/singularity-2.3.1  
for f in *.phy; do singularity exec $RAXMLNGIMG raxml-ng-mpi --msa $f --model JTT+G --parse; done`

####  Contents of raxml_all.sh

`#!/bin/bash`   

`#BSUB -n 8
#BSUB -R rusage[mem=2048]
#BSUB -W 8:00
#BSUB -q long
#BSUB -R "span[hosts=1]"
#BSUB -o "raxml_all_job.out"
#BSUB -e "raxml_all_job.err"`   

`module load raxml-ng/0.9.0
module load singularity/singularity-2.3.1
for f in *reduced.phy; do singularity exec $RAXMLNGIMG raxml-ng-mpi --msa $f --model JTT+G --threads 8; done`  


In [ ]:
#running RF analysis

#concatenate all the trees

In [ ]:
!

In [ ]:
 sed 's/OM[^:]*:/Omo:/g' test.tree

In [ ]:
#Madelaine's original scripts are below

In [ ]:
##check that there are no duplicate peptide libraries before going onto this step (use libraries called 'fixed' if there are 2)
##first, make all database proteins on 1 line (1line.sh)
for f in *.fasta; do awk '/^>/ {printf("\n%s\n",$0);next; } { printf("%s",$0);} END {printf("\n");}' > $fullfile.1line.fa; done

##search for all geneIDs in the peptide databases
##*.txt is each file in the 'LociForAlignment' folder

# amber note:is this generating a new fasta from the locus_x.txt with the sequence underneath each gene_id?
for file in *.txt; do 
	grep -A 1 -w -m 1 -f $file -h *.fa >${file}.fasta
done


rm *.txt
mv *.fasta LociForAlignment 

##get rid of -- that grep adds between hits
for file in *; do
	sed -i .bu.txt 's/--//g' $file
done

##remove backups
rm *.bu.txt

##on MGHPCC
for file in *; do sed -i 's/--//g' $file; done

In [ ]:
############################################################################################################################################
##STEP 3: align using MAFFT (L-in-si) (script = mafft_sln.sh)
##copied everything over to the MGHPCC. Using cyberduck (SFTP, port 22):
Server: ghpcc06.umassrc.org

##I use the interactive q for this because I'm too lazy to write a script
bsub -Is -q interactive -W 8:00 -n8 -R "span[hosts=1]" /bin/bash
./mafft_aln.sh


##align using MAFFT (L-in-si) (script = mafft_aln.sh)
files={{directory}}*.fa
for f in $files
	do
		fullfile=$f
		fasta_file=$(basename "$fullfile")
		echo "full in_file = "$fasta_file
		mafft-linsi $f >{{directory}}${fasta_file}.aln
	done



############################################################################################################################################
##STEP 3: replace gene names with species names (i.e. all arabidopsis genes should have the name Ath in every single gene file)

##will be different for each family, important to check output to make sure all the right changes being made
##make a 'Test' directory and play around in there before changing all files
##KEEP VERSIONS WITH ORIGINAL TAXON NAMES - OTHERWISE YOU WILL NEED TO REPEAT THE TIME-CONSUMING MAFFT ALIGNMENT

##Solanaceae
#RefGeneID,Nicotiana,Petunia,Phygri1.3.fixed,Jsinuosa,SMEbrinjal,Ppruinosa,Stuberosum,Schacoense,Spennellii,Cannuum,Smelongena,Slycoperscoides06
#Taxa: Slyc,Nic,Pet,Phygri,Jaltomata,Smel,Ppruin,Stube,Scha,Spenn,Cann,Slycopers

for file in *; do
	sed -i .3 's/>C.*/>Can/g; s/>Jalsin.*/>Jsi/g; s/>N.*/>Nic/g; s/>Pe.*/>Pet/g; s/>g.*/>Scha/g; s/>Solyc.*/>Slyc/g; s/>Solyd.*/>Slyd/g; s/>SMEL.*/>Smel/g; s/>SpL.*/>Spen/g; s/>PGSC.*/>Stub/g; s/>PHT.*/>Can/g; s/>Smech.*/>Brin/g; s/>Phygri.*/>Phygri/g; s/>Phypru.*/>Phypru/g' $file 
done


##Poaceae
##see 'Poaceae_Species.xlsx' for details
for file in *; do
	sed -i .3 's/>Aco.*/>Aco/g; s/>Ata.*/>Ata/g; s/>B.*/>Bdi/g; s/>KAF.*/>Dex/g; s/>OE.*/>Dol/g; s/>ELE.*/>Eco/g; s/>TV.*/>Ecu/g; s/>Et.*/>Ete/g; s/>H.*/>Hvu/g; s/>LP.*/>Lpe/g; s/>Mis.*/>Msi/g; s/>OB.*/>Obr/g; s/>OR.*/>Ogl/g; s/>Ola.*/>Ola/g; s/>KN.*/>Olo/g; s/>OM.*/>Ome/g; s/>Osat.*/>Osa/g; s/>Oro.*/>Oth/g; s/>Ph.*/>Pha/g; s/>Pah.*/>Pha/g; s/>Plat.*/>Pla/g; s/>Pav.*/>Pvi/g; s/>Rgu.*/>Rgu/g; s/>stran.*/>San/g; s/>Sobic.*/>Sbi/g; s/>SbR.*/>Sbi/g; s/>Seita.*/>Sit/g; s/>Sevi.*/>Svi/g; s/>Tra.*/>Tae/g; s/>Thint.*/>Tin/g; s/>TRI.*/>Tur/g; s/>Zm.*/>Zma/g;' $file; 
done

##Brassicaceae
#Taxa: Aal, Aha, Ali, Aly, Ath, Bol, Bra, Bst, Cam, Cgr, Chir, Chis, Cma, Cru, Cvi, Dso, Dst, Esa, Esy, Eve, Iam, Iti, Lan, Lsa, Mma, Mpe, Pis, Sal, Spa, Spi, Tar
##see 'Brassicaceae_Species.xlsx' for details

for file in *; do
	sed -i .3 's/>Aa_.*/>Aal/g; s/>Araha.*/>Aha/g; s/>Alyli.*/>Ali/g; s/>AL.*/>Aly/g; s/>AT.*/>Ath/g; s/>Bol.*/>Bol/g; s/>Brara.*/>Bra/g; s/>Bostr.*/>Bst/g; s/>Caamp.*/>Cam/g; s/>Cagra.*/>Cgr/g; s/>CARHR.*/>Chir/g; s/>Crahi.*/>Chis/g; s/>Camar.*/>Cma/g; s/>Carub.*/>Cru/g; s/>Clevi.*/>Cvi/g; s/>Desop.*/>Dso/g; s/>Distr.*/>Dst/g; s/>Thhalv.*/>Esa/g; s/>Eusyr.*/>Esy/g; s/>Eruve.*/>Eve/g; s/>Ibeam.*/>Iam/g; s/>Isati.*/>Iti/g; s/>Luann.*/>Lan/g; s/>Lesat.*/>Lsa/g; s/>Mamar.*/>Mma/g; s/>Myper.*/>Mpe/g; s/>Roisl.*/>Ris/g; s/>Sialb.*/>Sal/g; s/>Sp.*/>Spa/g; s/>Stapi.*/>Sta/g; s/>Thlar.*/>Tar/g;' $file; 
done

##on MGHPCC (don't need the '-i' switch - that's a mac osx thing)
for file in *; do sed -i 's/>Aa_.*/>Aal/g; s/>Araha.*/>Aha/g; s/>Alyli.*/>Ali/g; s/>AL.*/>Aly/g; s/>AT.*/>Ath/g; s/>Bol.*/>Bol/g; s/>Brara.*/>Bra/g; s/>Bostr.*/>Bst/g; s/>Caamp.*/>Cam/g; s/>Cagra.*/>Cgr/g; s/>CARHR.*/>Chir/g; s/>Crahi.*/>Chis/g; s/>Camar.*/>Cma/g; s/>Carub.*/>Cru/g; s/>Clevi.*/>Cvi/g; s/>Desop.*/>Dso/g; s/>Distr.*/>Dst/g; s/>Thhalv.*/>Esa/g; s/>Eusyr.*/>Esy/g; s/>Eruve.*/>Eve/g; s/>Ibeam.*/>Iam/g; s/>Isati.*/>Iti/g; s/>Luann.*/>Lan/g; s/>Lesat.*/>Lsa/g; s/>Mamar.*/>Mma/g; s/>Myper.*/>Mpe/g; s/>Roisl.*/>Ris/g; s/>Sialb.*/>Sal/g; s/>Sp.*/>Spa/g; s/>Stapi.*/>Sta/g; s/>Thlar.*/>Tar/g;' $file; done


##remove backups
rm *.3


##
##on MGHPCC (don't need the '-i' switch - that's a mac osx thing)
for file in *; do sed -i 's/>Aco.*/>Aco/g; s/>Ata.*/>Ata/g; s/>B.*/>Bdi/g; s/>KAF.*/>Dex/g; s/>OE.*/>Dol/g; s/>ELE.*/>Eco/g; s/>TV.*/>Ecu/g; s/>Et.*/>Ete/g; s/>H.*/>Hvu/g; s/>LP.*/>Lpe/g; s/>Mis.*/>Msi/g; s/>OB.*/>Obr/g; s/>OR.*/>Ogl/g; s/>Ola.*/>Ola/g; s/>KN.*/>Olo/g; s/>OM.*/>Ome/g; s/>Osat.*/>Osa/g; s/>Oro.*/>Oth/g; s/>Ph.*/>Pha/g; s/>Pah.*/>Pha/g; s/>Plat.*/>Pla/g; s/>Pav.*/>Pvi/g; s/>Rgu.*/>Rgu/g; s/>stran.*/>San/g; s/>Sobic.*/>Sbi/g; s/>SbR.*/>Sbi/g; s/>Seita.*/>Sit/g; s/>Sevi.*/>Svi/g; s/>Tra.*/>Tae/g; s/>Thint.*/>Tin/g; s/>TRI.*/>Tur/g; s/>Zm.*/>Zma/g;' $file; done
	
##replace * and . characters in fasta files (do after name replacement - otherwise you'll find all periods in the gene names)
	##to find them 
	grep -F '.' *.fasta.aln
	grep -F '*' *.fasta.aln

##mac replace them
for file in *; do 
	sed -i .3  's/\.//g' $file;
	sed -i .3  's/\*//g' $file;
done

##MGHPCC replace them
for file in *; do sed -i  's/\.//g' $file; done
for file in *; do sed -i  's/\*//g' $file; done

In [ ]:
############################################################################################################################################
##STEP 4: convert fasta alignments to phylip 

## f2p_run.sh is a wrapper for running Fasta2Phylip over many files (Fasta2Phylip.pl from https://github.com/josephhughes/Sequence-manipulation/blob/master/Fasta2Phylip.pl):
aln_files={{directory}}*.aln
for g in $aln_files
	do
		fullfile=$g
		aln_file=$(basename "$fullfile")
		echo "converting "$aln_file "to phylip"
		./Fasta2Phylip.pl $g >{{directory}}${aln_file}.phy
	done



##move all phylip alignments into the right folder
mv *.phy /home/mb90a/CNS/PhylipAlignments/

## STEP 5: ML tree estimation for each alignment

In [ ]:
#STEP 5: ML tree estimation for each alignment

##ML tree estimation on MGHPCC using raxml_batch.sh
bsub -q long -W 8:00 -n 8  -R "span[hosts=1]" -J [job name] -o [outfile].out <raxml_batch.sh ##explanation: https://www.umassrc.org/wiki/index.php/Submitting_Cluster_Jobs

		
bjobs 4649798 ##gets job status using job number (note it down when run starts)
bkill [job number] ##kills jobs

###ML tree estimation (./raxml_batch.sh) 
##!/bin/bash
#module load RAxML/8.2.11
#module load perl/5.18.1
#perl raxml_wrapper.pl -c config.raxml -i /home/mb90a/CNS/PhylipAlignments -o /home/mb90a/CNS/RAxML_Results



##concatenate all result files
cat RAxML_result.*.* >Family_IndividualLocusTrees.phy

## STEP 6: check topology of trees to find those loci to use in final analysis

In [ ]:
############################################################################################################################################
##STEP 6: check topology of trees to find those loci to use in final analysis

##run Robinson-Foulds on MGHPCC (robinson-foulds.sh)
module load RAxML/8.2.11
raxmlHPC-SSE3 -m GTRCAT -z Solanaceae_IndividualLocusTrees.phy -f r -n TEST.txt

bsub -q short -W 4:00 -n 8  -R "span[hosts=1]" -J poa_rf -o poa_rf.out <robinson-foulds.sh 

	##robinson-foulds analysis will fail if (1) there are trees with the wrong taxon names (i.e. renaming failed) OR if (2) there are trees with the wrong number of taxa (i.e. something went wrong with splitting files)

##this error means that there are trees in there with the wrong number of taxa (see check above for taxon numbers):
raxmlHPC-SSE3: bipartitionList.c:2103: computeRF: Assertion `tr->mxtips == tr->ntips' failed.
./robinson-foulds.sh: line 4: 195718 Aborted                 raxmlHPC-SSE3 -m GTRCAT -z Poaceae_IndividualLocusTrees.phy -f r -n Poaceae_rf.txt



##To find best trees, two options:
	#1: examine trees visually using FigTree to find a tree that matches known topology, use those that are close by Robinson-Foulds metric (pull full treefile in, go through them one at a time, need to unroot each time you switch between trees with <CMD> M)
	#2: make a constraint tree, add it to the top of the treefile before running robinson-foulds

##to make a constraint tree:
##look at phylogeny papers to figure out relationships
#helpful: https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=Root
##Make tree with the right topology using mesquite
#1 - make new file (file>new)
#2 - include tree file (Taxa&Trees>Import File With Trees>Include File)
#3 - click on tree, choose 'View Tree' - edit in tree window

##ended up having to unroot by hand (should be tritomy at root - get rid of a pair of brackets)




## STEP 7: use RF distances to identify loci to use downstream

In [ ]:
############################################################################################################################################
##STEP 7: use RF distances to identify loci to use downstream

##trees that are close to the constraint or tree chosen in #1 (RF distance small, will differ family by family) should be noted and listed in 'GoodTrees.csv'

##GoodTrees.csv should have one column (numbers of Tree2 to include downstream): TreeNumber

##NOTE - unix sort is different to mac osx sort. Trees will be in a different order in the file after cat command. To match tree and locus number, copy output of ls -l into a file, that will list them in the order cat would have added them, use Excel to give each tree a number (or something better when process more streamlined)
##TreeOrder.csv should have one column (numbers of Tree2 to include downstream): TreeNumber, LocusName

##Finding matching trees in R (143 Solanaceae trees have the right topology - RF distance <2? <1? need to check - determined with Excel)
library(dplyr)
order <- read.csv(file = "TreeOrder.csv")
good <- read.csv(file = "GoodTrees.csv")

good <- inner_join(good, order, by = "TreeNumber")
write.csv(good, file = "GoodTrees.csv") ##this over-writes GoodTrees.csv - call the original file something else if this bugs you

##make a list of all the loci (the LocusName column in GoodTrees.csv) in GoodLoci.txt

##selecting right tree files using GoodLoci.txt
for i in $(cat GoodLoci.txt); 
	do mv "$i" GoodLoci/; 
done




In [ ]:
############################################################################################################################################
##STEP 8: concatenate all good loci, run RAxML analysis

##concatenate all GoodLoci into a single alignment:
https://github.com/nylander/catfasta2phyml
./catfasta2phyml.pl -s [list of GoodLoci aln files] >> >Poaceae_GoodLoci.aln.phy

##for example:
./catfasta2phyml.pl -s locus_295.txt.fasta.aln locus_379.txt.fasta.aln locus_168.txt.fasta.aln locus_368.txt.fasta.aln locus_130.txt.fasta.aln locus_140.txt.fasta.aln locus_273.txt.fasta.aln locus_41.txt.fasta.aln locus_113.txt.fasta.aln locus_172.txt.fasta.aln locus_274.txt.fasta.aln locus_284.txt.fasta.aln locus_423.txt.fasta.aln locus_428.txt.fasta.aln locus_45.txt.fasta.aln locus_503.txt.fasta.aln locus_615.txt.fasta.aln locus_636.txt.fasta.aln locus_674.txt.fasta.aln locus_690.txt.fasta.aln locus_729.txt.fasta.aln locus_789.txt.fasta.aln locus_175.txt.fasta.aln locus_192.txt.fasta.aln locus_28.txt.fasta.aln locus_437.txt.fasta.aln locus_441.txt.fasta.aln locus_451.txt.fasta.aln locus_501.txt.fasta.aln locus_546.txt.fasta.aln locus_658.txt.fasta.aln locus_664.txt.fasta.aln locus_741.txt.fasta.aln locus_115.txt.fasta.aln locus_154.txt.fasta.aln locus_162.txt.fasta.aln locus_419.txt.fasta.aln locus_462.txt.fasta.aln locus_592.txt.fasta.aln locus_597.txt.fasta.aln locus_604.txt.fasta.aln locus_649.txt.fasta.aln locus_687.txt.fasta.aln locus_694.txt.fasta.aln locus_699.txt.fasta.aln locus_775.txt.fasta.aln locus_777.txt.fasta.aln locus_111.txt.fasta.aln locus_163.txt.fasta.aln locus_171.txt.fasta.aln locus_19.txt.fasta.aln locus_256.txt.fasta.aln locus_26.txt.fasta.aln locus_27.txt.fasta.aln locus_375.txt.fasta.aln locus_415.txt.fasta.aln locus_514.txt.fasta.aln locus_533.txt.fasta.aln locus_578.txt.fasta.aln locus_579.txt.fasta.aln locus_666.txt.fasta.aln locus_668.txt.fasta.aln locus_684.txt.fasta.aln locus_685.txt.fasta.aln locus_711.txt.fasta.aln locus_71.txt.fasta.aln locus_724.txt.fasta.aln locus_790.txt.fasta.aln locus_102.txt.fasta.aln locus_188.txt.fasta.aln locus_191.txt.fasta.aln locus_263.txt.fasta.aln locus_264.txt.fasta.aln locus_283.txt.fasta.aln locus_287.txt.fasta.aln locus_321.txt.fasta.aln locus_329.txt.fasta.aln locus_357.txt.fasta.aln locus_378.txt.fasta.aln locus_389.txt.fasta.aln locus_468.txt.fasta.aln locus_474.txt.fasta.aln locus_475.txt.fasta.aln locus_492.txt.fasta.aln locus_519.txt.fasta.aln locus_554.txt.fasta.aln locus_584.txt.fasta.aln locus_585.txt.fasta.aln locus_638.txt.fasta.aln locus_640.txt.fasta.aln locus_646.txt.fasta.aln locus_656.txt.fasta.aln locus_659.txt.fasta.aln locus_708.txt.fasta.aln locus_709.txt.fasta.aln locus_716.txt.fasta.aln locus_717.txt.fasta.aln locus_726.txt.fasta.aln locus_735.txt.fasta.aln locus_740.txt.fasta.aln locus_7.txt.fasta.aln locus_815.txt.fasta.aln locus_818.txt.fasta.aln locus_819.txt.fasta.aln locus_830.txt.fasta.aln locus_86.txt.fasta.aln locus_91.txt.fasta.aln locus_137.txt.fasta.aln locus_167.txt.fasta.aln locus_178.txt.fasta.aln locus_179.txt.fasta.aln locus_189.txt.fasta.aln locus_199.txt.fasta.aln locus_200.txt.fasta.aln locus_213.txt.fasta.aln locus_228.txt.fasta.aln locus_237.txt.fasta.aln locus_25.txt.fasta.aln locus_270.txt.fasta.aln locus_328.txt.fasta.aln locus_337.txt.fasta.aln locus_35.txt.fasta.aln locus_370.txt.fasta.aln locus_373.txt.fasta.aln locus_407.txt.fasta.aln locus_409.txt.fasta.aln locus_417.txt.fasta.aln locus_427.txt.fasta.aln locus_430.txt.fasta.aln locus_449.txt.fasta.aln locus_44.txt.fasta.aln locus_466.txt.fasta.aln locus_473.txt.fasta.aln locus_486.txt.fasta.aln locus_500.txt.fasta.aln locus_521.txt.fasta.aln locus_52.txt.fasta.aln locus_531.txt.fasta.aln locus_549.txt.fasta.aln locus_560.txt.fasta.aln locus_569.txt.fasta.aln locus_570.txt.fasta.aln locus_66.txt.fasta.aln locus_745.txt.fasta.aln locus_770.txt.fasta.aln locus_771.txt.fasta.aln locus_778.txt.fasta.aln locus_785.txt.fasta.aln locus_820.txt.fasta.aln locus_843.txt.fasta.aln locus_88.txt.fasta.aln >Poaceae_GoodLoci.aln.phy

##convert to phylip, then ran with raxml wrapper on mghpcc (raxml_concatenated.sh)
./Fasta2Phylip.pl Solanaceae_GoodLoci_Concatenated.aln.fa 


############################################################################################################################################
##NOTES
##for downloaded packages - had to run chmod and remove apple quarantine flags from them. instructions here:
https://www.alansiu.net/2021/08/19/troubleshooting-zsh-operation-not-permitted/
xattr -l catfasta2phyml.pl
xattr -d com.apple.quarantine catfasta2phyml.pl

##can also work when scripts won't run
chmod 777 script.sh

##fixed figtree with these instructions:
https://github.com/rambaut/figtree/issues/173

##this Java application stub:
https://github.com/tofi86/universalJavaApplicationStub/releases/

##for editing text files in the terminal, I use vim
vim filename.csv
#Instructions: https://vim.rtorr.com/